In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")
import numpy as np
import pandas as pd
import random
from collections import deque
from sklearn import preprocessing
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
SEQ_LEN = 60 
FUTURE_PERIOD_PREDICT = 3  
RATIO_TO_PREDICT = "LTC-USD"
BATCH_SIZE = 64
EPOCHS = 10
NAME = f"{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}"

In [ ]:
def classify(current, future):
    if float(future) > float(current):  
        return 1
    else: 
        return 0

In [ ]:
def preprocess_df(df):
    df = df.drop("future", 1)

    for col in df.columns:  
        if col != "target":  
            df[col] = df[col].pct_change()  
            df.dropna(inplace=True)  
            df[col] = preprocessing.scale(df[col].values)  

    df.dropna(inplace=True) 


    sequential_data = [] 
    prev_days = deque(maxlen=SEQ_LEN) 

    for i in df.values:  
        prev_days.append([n for n in i[:-1]]) 
        if len(prev_days) == SEQ_LEN:  
            sequential_data.append([np.array(prev_days), i[-1]])  

    random.shuffle(sequential_data)

    buys = [] 
    sells = [] 

    for seq, target in sequential_data:  
        if target == 0:  
            sells.append([seq, target])  
        elif target == 1:  
            buys.append([seq, target])  

    random.shuffle(buys)  
    random.shuffle(sells)  

    lower = min(len(buys), len(sells))  

    buys = buys[:lower]  
    sells = sells[:lower] 

    sequential_data = buys+sells  
    random.shuffle(sequential_data)  

    X = []
    y = []

    for seq, target in sequential_data: 
        X.append(seq)  
        y.append(target)  

    return np.array(X), y  


In [ ]:
main_df = pd.DataFrame() 

ratios = ["BTC-USD", "LTC-USD", "BCH-USD", "ETH-USD"]  
for ratio in ratios:  

    ratio = ratio.split('.csv')[0]  
    dataset = f'/content/gdrive/My Drive/crypto_data/{ratio}.csv' 
    df = pd.read_csv(dataset, names=['time', 'low', 'high', 'open', 'close', 'volume']) 

    df.rename(columns={"close": f"{ratio}_close", "volume": f"{ratio}_volume"}, inplace=True)

    df.set_index("time", inplace=True) 
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]  

    if len(main_df)==0: 
        main_df = df 
    else:  
        main_df = main_df.join(df)

main_df.fillna(method="ffill", inplace=True) 
main_df.dropna(inplace=True)


main_df['future'] = main_df[f'{RATIO_TO_PREDICT}_close'].shift(-FUTURE_PERIOD_PREDICT)
main_df['target'] = list(map(classify, main_df[f'{RATIO_TO_PREDICT}_close'], main_df['future']))

main_df.dropna(inplace=True)


times = sorted(main_df.index.values)
last_5pct = sorted(main_df.index.values)[-int(0.05*len(times))]

validation_main_df = main_df[(main_df.index >= last_5pct)]
main_df = main_df[(main_df.index < last_5pct)]

train_x, train_y = preprocess_df(main_df)
validation_x, validation_y = preprocess_df(validation_main_df)


print(f"train data: {len(train_x)} validation: {len(validation_x)}")
print(f"Dont buys: {train_y.count(0)}, buys: {train_y.count(1)}")
print(f"VALIDATION Dont buys: {validation_y.count(0)}, buys: {validation_y.count(1)}")
train_x = np.asarray(train_x)
train_y = np.asarray(train_y)
validation_x = np.asarray(validation_x)
validation_y = np.asarray(validation_y)

train data: 77922 validation: 3860
Dont buys: 38961, buys: 38961
VALIDATION Dont buys: 1930, buys: 1930


In [ ]:
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())  #normalizes activation outputs, same reason you want to normalize your input data.

model.add(LSTM(128, activation='relu', return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(LSTM(128, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)

# Compile model
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)

history = model.fit(
    train_x, train_y,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(validation_x, validation_y)
)

score = model.evaluate(validation_x, validation_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save("models/{}".format(NAME))

Epoch 1/10
1218/1218 [==============================] - 507s 413ms/step - loss: 0.7170 - accuracy: 0.5131 - val_loss: 0.9249 - val_accuracy: 0.4982
Epoch 2/10
1218/1218 [==============================] - 500s 410ms/step - loss: 0.6921 - accuracy: 0.5199 - val_loss: 0.6938 - val_accuracy: 0.5202
Epoch 3/10
1218/1218 [==============================] - 500s 410ms/step - loss: 0.6911 - accuracy: 0.5245 - val_loss: 0.6936 - val_accuracy: 0.5080
Epoch 4/10
1218/1218 [==============================] - 497s 408ms/step - loss: 0.6910 - accuracy: 0.5249 - val_loss: 0.6940 - val_accuracy: 0.5054
Epoch 5/10
1218/1218 [==============================] - 509s 418ms/step - loss: 0.6905 - accuracy: 0.5266 - val_loss: 0.6943 - val_accuracy: 0.5028
Epoch 6/10
1218/1218 [==============================] - 500s 410ms/step - loss: 0.6903 - accuracy: 0.5282 - val_loss: 0.6926 - val_accuracy: 0.5225
Epoch 7/10
1218/1218 [==============================] - 494s 406ms/step - loss: 0.6929 - accuracy: 0.5084 - val_